In [1]:
import cv2 as cv
from pyzbar import pyzbar
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Định nghĩa bộ lọc 3x3 trung vị của ảnh
def medianImg3x3(img):
    m, n = img.shape
    img_new = np.zeros_like(img)
    for i in range(1, m-1):
        for j in range(1, n-1):
            temp = [img[i-1, j-1], img[i-1, j], img[i-1, j + 1],
                    img[i, j-1], img[i, j], img[i, j + 1],
                    img[i + 1, j-1], img[i + 1, j], img[i + 1, j + 1]]

            temp = sorted(temp)
            img_new[i, j] = temp[4]

    return img_new

In [3]:
# Filter làm nét ảnh
def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    
    sharpened_image = cv.filter2D(image, -1, kernel)
    
    return sharpened_image

In [4]:
def view(qrCodes_processed, image, img):    
    for qrCode in qrCodes_processed:
        (x, y, w, h) = qrCode.rect
        cv.rectangle(image, (x, y), (x+w, y+h), (0, 255, 255), 2)
        qrCodeData = qrCode.data.decode("utf-8")
        qrCodeType = qrCode.type
        text = "{} - {}".format(qrCodeData, qrCodeType)
        print(text)
        cv.putText(image, text, (x-10, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    cv.imshow("Phat hien QR code", image)  
    cv.imshow("Anh goc", img) 

    key = cv.waitKey(0)

    if key == ord('q'):
        cv.destroyAllWindows()


In [5]:
def detect_qr_from_image(image_path):
    image = cv.imread(image_path)
    if image is None:
        print("Không thể đọc ảnh từ đường dẫn được cung cấp.")
        return

    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    qrCodes = pyzbar.decode(image)

    if len(qrCodes) == 0:
        print("Không tìm thấy mã QR trong ảnh gốc.")

        # Cân bằng sáng
        # clahe = cv.createCLAHE(clipLimit=15.0, tileGridSize=(35, 35))
        clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        equalized_gray = clahe.apply(gray)
        # equalized_gray = decrease_brightness(equalized_gray, 1)
        # cv.imshow("Anh goc", equalized_gray) 
        qrCodes = pyzbar.decode(equalized_gray)
        if len(qrCodes) == 0:
            print("Không phát hiện được mã QR trên ảnh sau khi được xử lý bằng cân bằng histogram")
            # Xử lý ảnh bằng medianImg3x3
            processed_image = medianImg3x3(gray)
            # Phát hiện mã QR trên ảnh đã được làm mờ
            qrCodes = pyzbar.decode(processed_image)

            if len(qrCodes) == 0:
                print("Không phát hiện được mã QR trên ảnh sau khi được xử lý bằng cân bằng trung vị")
                sharpen = sharpen_image(gray)
                qrCodes = pyzbar.decode(sharpen)
                if len(qrCodes) == 0:
                    print("Không phát hiện được mã QR trên ảnh sau khi được xử lý bằng bộ lọc làm nét")
                else:
                    print("Phát hiện mã QR trong ảnh đã được làm nét.")
                    view(qrCodes, sharpen,image)
            else:
                print("Phát hiện mã QR trong ảnh đã được xử lý khử nhiễu.")
                view(qrCodes, processed_image,image)
                
        else:
            print("Phát hiện mã QR trong ảnh đã được xử lý cân bằng sáng.")
            view(qrCodes, equalized_gray, image)
            

    else:
        print("Phát hiện mã QR trong ảnh gốc.")
        view(qrCodes, image, image)


In [6]:
detect_qr_from_image("./Img/case8.png")
#  nhấn "q" để tắt cửa sổ cv.imshow
# case 1 ảnh bình thường
# case 4 ảnh nhiễu
# case 5 ảnh tối
# case 8 ảnh mờ
# case 9 ảnh không có gì

Không tìm thấy mã QR trong ảnh gốc.
Không phát hiện được mã QR trên ảnh sau khi được xử lý bằng cân bằng histogram
Không phát hiện được mã QR trên ảnh sau khi được xử lý bằng cân bằng trung vị
Phát hiện mã QR trong ảnh đã được làm nét.
TEAM HOI DAP IS DA BEST! - QRCODE


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/hoang/.local/lib/python3.10/site-packages/cv2/qt/plugins"
